In [3]:
import pandas as pd

lyrics = pd.read_csv("lyrics_cleaned_desc_by_count.csv")
songs = pd.read_csv("merged_dropdup.csv")

In [4]:
#the function to correct spellings, make exception for common contractioni words
import re
def words(text): return re.findall(r'\w+', text.lower())
from collections import Counter
WORDS = Counter(words(open('big.txt').read()))
####add additional words to keep "gonna", "wanna", etc.
additional_words = pd.read_csv('contraction.txt', sep='\t', header=None)[0].tolist()[:-2]
additional_words = list(map(lambda w: w.split()[0], additional_words))
additional_words.append('wanna')
WORDS.update(additional_words)


def correct_spelling(word):

    def P(word, N=sum(WORDS.values())): 
        "Probability of `word`."
        return WORDS[word] / N  
    
    def correction(word): 
        "Most probable spelling correction for word."
        return max(candidates(word), key=P)

    def candidates(word): 
        "Generate possible spelling corrections for word."
        return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

    def known(words): 
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in words if w in WORDS)

    def edits1(word):
        "All edits that are one edit away from `word`."
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(word): 
        "All edits that are two edits away from `word`."
        return (e2 for e1 in edits1(word) for e2 in edits1(e1))
    
    return correction(word)

In [ ]:
#sort the lyrics by year, for a given genre
#only keep the words that are in the common word txt
#Since there are too few songs before 1978 and that there are only 3 songs in 2010, we will only look at 
#years between 1988 and 2009

In [5]:
#tag if word is in profanity word list
bad_words =  set(open('bad_words.txt').read().split())
def is_bad_word(word):
    if word in bad_words:
        return 1
    else:
        return 0

In [6]:
#sort the lyrics by year, for a given genre
#only keep the words that are in the common word txt
#Since there are too few songs before 1978 and that there are only 3 songs in 2010, we will only look at 
#years between 1988 and 2009
def by_year(genre):
    genre_popular = songs[(songs['genre_merged'] == genre) & (songs['artist_familiarity']>=0.5) & 
                          (songs['year'] > 1987) & (songs['year'] < 2010)]
    #locate the rap songs in the lyrics dataset
    #use the track_id as the key 
    genre_lyrics = lyrics.merge(genre_popular, left_on="track_id", right_on="track_id", how='inner')
    genre_lyrics['word_corrected'] = genre_lyrics['word'].map(lambda x: correct_spelling(x))
    #genre_lyrics = genre_lyrics[genre_lyrics['word_corrected'].isin(set(open('common_words_across_genres.txt').read().split()))]
    genre_by_year = genre_lyrics[['track_id', 'year', 'genre_merged', 'word_corrected','count']].sort_values(by='year')
    return genre_by_year

In [ ]:
rap_trend = by_year('rap')
rock_trend = by_year('rock')
print('rock done')
blues_trend = by_year('folk')
print('folk done')
indie_trend = by_year('indie')
print('indie done')
country_trend = by_year('country')
print('country done')
metal_trend = by_year('metal')
print('metal done')

In [121]:
#split years
#add columns: is_bad_words, word_length
#keep 100:150 words

def split_year(genre):
    saved_trends = dict()
    for i in range(1988, 2010):
        tmp = genre.loc[genre['year'] == i]
        tmp = tmp.groupby('word_corrected').sum().sort_values(by='count', ascending=False).reset_index()
        total = sum(tmp['count'])
        tmp['frequency'] = tmp['count']/total
        tmp['is_bad_word'] = tmp['word_corrected'].map(lambda x: is_bad_word(x))
        tmp['word_length'] = tmp['word_corrected'].map(lambda x: len(x))
        tmp['year'] = i
        tmp = tmp.loc[100:150]
        saved_trends[i] = tmp
    return saved_trends

In [ ]:
#each genre into a dict, each key corresponding to each year
rock_by_year_dict = split_year(rock_trend)
rap_by_year_dict = split_year(rap_trend)
blues_by_year_dict = split_year(blues_trend)
indie_by_year_dict = split_year(indie_trend)
country_by_year_dict = split_year(country_trend)
metal_by_year_dict = split_year(metal_trend)

In [128]:
#concatenate the 22 years for each year
#concatenate the dataframes
def concat_year(genre):
    frames = []
    for i in range(0, 22):
        frames.append(genre[(list(genre.keys())[i])])
    return pd.concat(frames)

In [ ]:
df_rock = concat_year(rock_by_year_dict)
df_rap = concat_year(rock_by_year_dict)
df_indie = concat_year(rock_by_year_dict)
df_metal = concat_year(rock_by_year_dict)
df_country = concat_year(rock_by_year_dict)
df_blues = concat_year(rock_by_year_dict)

In [146]:
df_rock['genre'] = 'rock'
df_rap['genre'] = 'rap'
df_indie['genre'] = 'indie'
df_metal['genre'] = 'metal'
df_country['genre'] = 'country'
df_blues['genre'] = 'blues'



In [154]:
df_grand = pd.concat([df_rock, df_rap, df_indie, df_metal, df_country, df_blues])

In [155]:
df_grand.to_csv('visual.csv', index=False)

In [157]:
pd.read_csv('visual.csv')

,word_corrected,year,count,frequency,is_bad_word,word_length,genre
0,lie,1988,22,0.002163,0,3,rock
1,around,1988,22,0.002163,0,6,rock
2,left,1988,22,0.002163,0,4,rock
3,better,1988,21,0.002065,0,6,rock
4,die,1988,21,0.002065,0,3,rock
5,smoke,1988,21,0.002065,0,5,rock
6,smell,1988,21,0.002065,0,5,rock
7,guess,1988,21,0.002065,0,5,rock
8,winter,1988,21,0.002065,0,6,rock
9,sing,1988,21,0.002065,0,4,rock


In [1]:
df_grand

NameError: name 'df_grand' is not defined

In [71]:
list(rap_trend_all.columns)

['word_corrected', 'year', 'count', 'frequency', 'is_bad_word', 'word_length']

# Make the plot

In [72]:
#make year range
years = []
for i in range(1988, 2010):
    years.append(str(i))

In [28]:
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML

init_notebook_mode(connected=True)

In [33]:
dataset = rap_trend_all

#make year range
years = []
for i in range(1987, 2010):
    years.append(str(i))

# make list of 50 top words
top_words = []
for continent in dataset['continent']:
    if continent not in continents:
        continents.append(continent)

['1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010']

In [ ]:


# make figure
figure = {
    'data': [],
    'layout': {},
    'frames': []
}

# fill in most of layout
figure['layout']['xaxis'] = {'range': [0, 0.001], 'title': 'Word Length'}
figure['layout']['yaxis'] = {'title': 'Is Bad Word or Not'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '1988',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

# make data
year = 1988
for continent in continents:
    dataset_by_year = dataset[dataset['year'] == year]
    dataset_by_year_and_cont = dataset_by_year[dataset_by_year['continent'] == continent]

    data_dict = {
        'x': list(dataset_by_year_and_cont['lifeExp']),
        'y': list(dataset_by_year_and_cont['gdpPercap']),
        'mode': 'markers',
        'text': list(dataset_by_year_and_cont['country']),
        'marker': {
            'sizemode': 'area',
            'sizeref': 200000,
            'size': list(dataset_by_year_and_cont['pop'])
        },
        'name': continent
    }
    figure['data'].append(data_dict)
    
# make frames
for year in years:
    frame = {'data': [], 'name': str(year)}
    for continent in continents:
        dataset_by_year = dataset[dataset['year'] == int(year)]
        dataset_by_year_and_cont = dataset_by_year[dataset_by_year['continent'] == continent]

        data_dict = {
            'x': list(dataset_by_year_and_cont['lifeExp']),
            'y': list(dataset_by_year_and_cont['gdpPercap']),
            'mode': 'markers',
            'text': list(dataset_by_year_and_cont['country']),
            'marker': {
                'sizemode': 'area',
                'sizeref': 200000,
                'size': list(dataset_by_year_and_cont['pop'])
            },
            'name': continent
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

    
figure['layout']['sliders'] = [sliders_dict]

iplot(figure)